<a href="https://colab.research.google.com/github/Rompil/hands-on/blob/master/PySpark%20Hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache-mirror.rbc.ru/pub/apache/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xvzf spark-2.4.1-bin-hadoop2.7.tgz.1
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Практическое введение в PySpark для Data Scientists

---

by Roman Pilyugin

## Содержание

---


1. [Немного теории. RDD, Transformation, Action](#theory)
2. [Если я уже умею в Python и Pandas](#forSkilledPeople)
3. [Сравнение операций Pandas и PySpark](#PandasVsPySpark)
    1. [Чтение данных](#csvRead)
    3. [Некоторые основные примеры манипуляций с данными](#moreOps)
    4. [Визуализация данным](#pics)
    5. [Сохранение данных](#storeData)
4. [Погружаемся в Machine Learning](#ML)

##  Немного теории. RDD, Transformation, Action <a name =theory></a>

**RDD** (Resilient Distributed Dataset) - Это фундаментальная абстракция структуры данных в Spark. Ее можно представить в виде неизменяемой коллекции объектов. Сама коллекция может быть размещена по нескольким нодам (отдельным машинам) для параллельной обработки.

**Transformation** - операция, результатом которой является новый RDD. Таким образром, transformation задает связь ( последовательность) между RDD в виде последовательности преобразований. 

*Надо отметить, что все операции Transformation выполняются в **Lazy mode**, т.е. пока не вызвать одну из оперций Action, то преобразования не выполняются.*

### Основные операции Transformation

* map(function) — *применяет функцию function к каждому элементу датасета*

* .filter(function) — *возвращает все элементы датасета, на которых функция function вернула истинное значение*

* .distinct([numTasks]) — *возвращает датасет, который содержит уникальные элементы исходного датасета*

**Также стоит отметить об операциях над множествами, смысл которых понятен из названий:**

* .union(otherDataset)

* .intersection(otherDataset)

* .cartesian(otherDataset) — *новый датасет содержит в себе всевозможные пары (A,B), где первый элемент принадлежит исходному датасету, а второй — датасету-аргументу*





**Action** - операции, результат которых появляется немедленно, в отличии от Transformation. Это могут быть операции для вывода результата вычислений, сохранения в отдельный файл или выгрузка во внешнее хранилище.
*Можно сказать, что Action операция опзволяет извлечь данные из RDD.*

### Основные операции Action


* .saveAsTextFile(path) — *сохраняет данные в текстовый файл (в hdfs, на локальную машину или в любую другую поддерживаемую файловую систему — полный список можно посмотреть в документации)*

* .collect() — *возвращает элементы датасета в виде массива. Как правило, это применяется в случаях, когда данных в датасете уже мало (применены различные фильтры и преобразования) — и необходима визуализация, либо дополнительный анализ данных, например средствами пакета Pandas*

* .take(n) — *возвращает в виде массива первые n элементов датасета*

* .count() — *возвращает количество элементов в датасете*

* .reduce(function) — *знакомая операция для тех, кто знаком с MapReduce. Из механизма этой операции следует, что функция function (которая принимает на вход 2 аргумента возвращает одно значение) должна быть обязательно коммутативной и ассоциативной*



## Если я уже умею в Python и Pandas <a name=forSkilledPeople></a>

<p>Для практикующего специалиста по анализу данных PySpark может быть полезен когда: </p>


*   Уже знакомы c Python (Scala, Java, R) и Pandas DataFrame
*   Есть возможность развернуть кластер для обработки данных
*   Нужно обрабатывать больше объемы данных [^1].

[^1]:   Под большими данными понимаются те, что не помещаются в оперативной памяти одного компьютера



## Сравнение операций Pandas и PySpark <a name=PandasVsPySpark></a>

### Чтение данных<a name=csvRead></a>
В PySpark можно читать данные непосредственно из notebook`а

```python
simpleData = [5,7,1,12,10,25]
newRDD = sc.parallelize(localData)
```
Но чаще всего приходится иметь дело с внешними источниками данных, которые поддерживает Hadoop - HDFS, HBase и т.д.
```python
alsoRDD = sc.textFile("path_to_your_data_file")
```
Замечу, что в обоих случаях мы получаем RDD, о котором мы говорили раньше и с ними уже доступны операции Transformation и Action. 

Чтобы получить что-то более привычное, типа DataFrame можно выполнить:


In [0]:
instant_df = spark.sparkContext.parallelize([(1, 2, 3, 'a b c'),\
                                     (4, 5, 6, 'd e f'),\
                                     (7, 8, 9, 'g h i')]).toDF(['col1', 'col2', 'col3','col4'])
instant_df.show() # Only now all transformations are executed. .show() - Action method

Или даже так:

In [0]:
Employee = spark.createDataFrame([('1', 'Joe', '70000', '1'),\
                                  ('2', 'Henry', '80000', '2'),\
                                  ('3', 'Sam', '60000', '2'),\
                                  ('4', 'Max', '90000', '1')],\
                                 ['Id', 'Name', 'Sallary','DepartmentId'])
Employee.show()


После первого знакомства с DataFrame в PySpark мы мможем сравнить с DataFrame в Pandas.

Ну и для первого примера рассмотрим чтение данных из CSV-файла.

__Как это в Pandas__
```python
import pandas as pd
df = pd.read_csv('/FileStore/tables/gt_sales_by_pos_month.csv')
```
Конечно, есть дополнительные параметры для детальнойго указания параметров CSV-файла и я их опустил.

__В PySpark__

In [0]:
df = spark.read.options(header=True)\ #  the first line of files are used to name columns and are not included in data.
                .options(inferSchema=True)\ # automatically infer column types. It requires one extra pass over the data and is false by default.
                .csv('gt_sales_by_pos_month.csv')

Подробнее про чение файлов и возможные опции в [документации](https://docs.databricks.com/spark/latest/data-sources/read-csv.html#reading-files) 

Более того, DataFrame во многом схожа с таблицей в базе данных и можно посмотреть её схему:

In [0]:
df.printSchema()

### Манипуляции с данными <a name=manipulation></a>

Взглянем на результат предыдущей операции.
Для этого можем посмотреть на первые строки нашего DataFrame.

__Как это в Pandas__
```python
df.head(5)
```

__В PySpark__

In [0]:
df.show()

In [0]:
df.show(5)

В остальном, некоторые операции практически идентичны.

In [0]:
df.columns # узнаем про столбцы в таблице

In [0]:
df.dtypes # узнаем про типы данных в таблице

In [0]:
df.describe().show() # .show() needs to be call to show results

**Выбор данных.**

В PySpark DataFrame нет индексов для строк и выбор строк осуществляется по заданию определенных условий- фильтров. Этоработает аналогично тому, как в pandas.Dataframe.

__Фильтрация__
Для выбора данных из Dataframe можно использовать операции сравнения, аналогично тому, как с pandas.dataframe.

In [0]:
df[df.meg_mrsp > 150].describe().show()

Таким образом можно формировать сложные уловия выбора:

In [0]:
df[(df.meg_mrsp > 85)&(df.brand_variant=='JADE LA ROSE 100 SSL')].count() # to know how many items satisfy the condition

**Операции со столбцами. **

__Удаление столбцов__

Может понадобиться убрать некоторые столбцы из таблицы т.к. они нам не пригодятся в дальнейшем.
*order_number* - отличный кандидат для этого.

__В Pandas__
```python
df.drop('order_number', axis=1)
```

__В PySpark__

In [0]:
df.drop('order_number').show(5)

В PySpark нет понятия индекса и axis. Вам не надо указывать явную ось.

__Добавление столбцов__.

DataFrame неизменяемы т.е. нельзя изменить содержимое столбца, но можно создать новый DataFrame.

In [0]:
df.select('brand_manufacturer').distinct().show()